In [ ]:
import pandas as pd
import urllib

url = "https://data.smgov.net/api/views/ng8m-khuz/rows.csv?accessType=DOWNLOAD"
historical = "./data.csv"
static = "./static.csv"

In [ ]:
# refresh the local file of historical data
urllib.urlretrieve(url, historical)

In [ ]:
# read files into DataFrames
hdf = pd.read_csv(historical)
sdf = pd.read_csv(static)

In [ ]:
# clean up addresses in historical data to match those in static data
hdf.Address = hdf.Address.replace(
    [r", Santa Monica", r"\.", r"\bSt\b", r"\sat 4th Street"],
    [               "",    "",  "Street",                 ""],
    regex = True
)

In [ ]:
# merge the DataFrames on the Address columns
# this is an 'inner join' => any Addresses missing in the static data
# will have their corresponding historical records removed
# then select out only the columns we are interested in
df = hdf.merge(sdf, on = "Address")[[
    "Date/Time",
    "Address",
    "Zip",
    "Latitude",
    "Longitude",
    "Lot",    
    "Available",
    "Total Spaces"
]]

In [ ]:
# renaming columns
df.rename(
    columns = { "Date/Time": "DateTime", "Total Spaces": "Total" },
    inplace = True
)

In [ ]:
# convert strings to datetime64[ns, America/Los_Angeles] objects
df.DateTime = pd.to_datetime(df.DateTime, format = "%m/%d/%Y %I:%M:%S %p")
df.DateTime = df.DateTime.dt.tz_localize("America/Los_Angeles")

In [ ]:
# fix instances where there are more available than total spaces
mask = df.Available > df.Total
df.loc[mask, "Available"] = df.loc[mask, "Total"]

In [ ]:
# calc in an availability score
df["Availability"] = df.Available / df.Total